In [ ]:
import re
import pandas as pd

In [ ]:
def label_words(text):
    """Размечает слова в предложении на O (other) и D (dialect)."""
    tokens = []
    words = re.split(r'(\[.*?\]|\s+)', text)  # Разделяем текст на слова и пробелы, включая пометки []

    for word in words:
        if word.startswith('[') and word.endswith(']'):
            clean_word = word[1:-1]  # Убираем квадратные скобки
            tokens.append((clean_word, 'D'))
        elif word.strip():  # Если слово не пустое (не пробел)
            tokens.append((word.strip(), 'O'))

    return tokens

In [ ]:
def is_invalid_corrected(text):
    """Проверяет, является ли текст недопустимым (только многоточия или только цифры)."""
    # Удаляем пробелы и проверяем на содержание только цифр
    if re.fullmatch(r'\d+', text.strip()):
        return True
    # Проверяем, состоит ли текст только из многоточий
    if text.strip() == '......':
        return True
    return False

In [ ]:
def process_highlighted_errors(input_file, output_file):
    # Шаг 1: Прочитать файл
    df = pd.read_csv(input_file)

    # Шаг 2: Убедиться, что в колонке 'highlighted' нет NaN значений
    df = df.dropna(subset=['highlighted', 'corrected'])

    # Шаг 3: Отфильтровать строки, где есть квадратные скобки []
    df_with_errors = df[df['highlighted'].str.contains(r'\[.*?\]')]

    # Шаг 4: Удалить строки, где в колонке 'corrected' содержится "ошибки в тексте"
    df_with_errors = df_with_errors[~df_with_errors['corrected'].str.contains("ошибки в тексте")]

    # Шаг 5: Удалить строки, где 'corrected' состоит только из многоточий или только из цифр
    df_with_errors = df_with_errors[~df_with_errors['corrected'].apply(is_invalid_corrected)]

    # Шаг 6: Разметить слова
    df_with_errors['labeled'] = df_with_errors['highlighted'].apply(label_words)

    # Шаг 7: Записать результат в файл
    df_with_errors.to_csv(output_file, index=False)

    # Возврат отфильтрованного и размеченного DataFrame
    return df_with_errors

In [ ]:
# Использование функции
df_with_errors = process_highlighted_errors('highlighted_errors.csv', 'df_with_errors.csv')